In [ ]:
!nvidia-smi

Fri Sep 20 12:51:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Calculando a variância usando iteradores dinâmicos**

In [18]:
%%writefile exercicio1.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;
#include <thrust/iterator/constant_iterator.h>


int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::device_vector<double> apple;

  thrust::device_vector<double> microsoft;

  thrust::device_vector<double> diferenca;


  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {

                stringstream ss(linha);
                string parte1, parte2;

                getline(ss, parte1, ',');
                getline(ss, parte2, ',');

                double valor1 = stod(parte1);
                double valor2 = stod(parte2);
                apple.push_back(valor1);
                microsoft.push_back(valor2);

                double valor_diferenca;
                valor_diferenca = valor1 - valor2;

                diferenca.push_back(valor_diferenca);

                n = n + 1;

            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  double preco_medio_diferenca = thrust::reduce(diferenca.begin(), diferenca.end()) / n;

    thrust::constant_iterator<double> media(preco_medio_diferenca);

    double soma_diferencas_quadradas = 0.0;

    for (int i = 0; i < n; ++i) {
        double diferenca_valor = diferenca[i];
        double valor_quadrado = (diferenca_valor - preco_medio_diferenca) * (diferenca_valor - preco_medio_diferenca);
        soma_diferencas_quadradas += valor_quadrado;
    }

    double variancia = soma_diferencas_quadradas / n;

    cout << "Preço médio das diferenças: " << preco_medio_diferenca << endl;
    cout << "Variância das diferenças: " << variancia << endl;

    return 0;
}



Overwriting exercicio1.cu


In [20]:
!nvcc -arch=sm_70 -std=c++14 exercicio1.cu -o exercicio1

In [21]:
!./exercicio1

stocks2.txt
stocks2.txtPreço médio das diferenças: -265.232
Variância das diferenças: 16120.3


**Análise de Variação Diária dos Preços de Ações**

In [22]:
%%writefile exercicio2.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::device_vector<double> stocks;

  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {
                double valor = stod(linha);
                stocks.push_back(valor);
                n = n +1;
            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  thrust::device_vector<double> ganho_diario(stocks.size() - 1);

  thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), thrust::minus<double>());



  return 0;

}

Writing exercicio2.cu


In [23]:
!nvcc -arch=sm_70 -std=c++14 exercicio2.cu -o exercicio2

In [24]:
!./exercicio2

stocks-google.txt
stocks-google.txt

**Contagem de Dias com Aumento no Preço das Ações¶**

In [34]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
#include <thrust/count.h>

using namespace std;

int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::device_vector<double> stocks;

  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {
                double valor = stod(linha);
                stocks.push_back(valor);
                n = n +1;
            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  thrust::device_vector<double> ganho_diario(stocks.size() - 1);

  thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), thrust::minus<double>());

  int quantos_subiu = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), [] __device__(int x){
    return x > 0;
  });

  cout << "Dias que aumentou: " << quantos_subiu  << endl;

  return 0;

}

Overwriting exercicio3.cu


In [38]:
!nvcc -arch=sm_70 -std=c++14 --extended-lambda exercicio3.cu -o exercicio3

In [39]:
!./exercicio3

stocks-google.txt
stocks-google.txtDias que aumentou: 55


**Cálculo do Aumento Médio nos Dias em que o Preço Subiu¶**

In [45]:
%%writefile exercicio4.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
#include <thrust/count.h>
#include <thrust/replace.h>


using namespace std;

struct is_less_than_zero
{
  __host__ __device__
  bool operator()(int x)
  {
    return x < 0;
  }
};

int main(){

  string nome_arquivo;
  cin >> nome_arquivo;

  cout << nome_arquivo;

  ifstream entrada(nome_arquivo);

  if (!entrada.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

  thrust::device_vector<double> stocks;

  string linha;

  int n = 0;

  while (getline(entrada, linha)) {
        try {
            if (!linha.empty()) {
                double valor = stod(linha);
                stocks.push_back(valor);
                n = n +1;
            }
        } catch (const std::invalid_argument& e) {
            cerr << "Linha inválida encontrada: '" << linha << "'" << endl;
        }
  }

  entrada.close();

  thrust::device_vector<double> ganho_diario(stocks.size() - 1);

  thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), thrust::minus<double>());

  int quantos_subiu = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), [] __device__(int x){
    return x > 0;
  });

  cout << "Dias que aumentou: " << quantos_subiu  << endl;

  is_less_than_zero pred;

  thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), pred, 0);

  double soma = 0;

  int n_positivo = 0;

  for (int i = 0; i < n-1; i++){
    if (ganho_diario[i] > 0){
      n_positivo = n_positivo + 1;
    }
    soma =  soma + ganho_diario[i];
  }
  soma = soma/n_positivo;

  cout << "Aumento médio dos dias em que subiu: " << soma << endl;

  return 0;

}

Overwriting exercicio4.cu


In [46]:
!nvcc -arch=sm_70 -std=c++14 --extended-lambda exercicio4.cu -o exercicio4

In [47]:
!./exercicio4

stocks-google.txt
stocks-google.txtDias que aumentou: 55
Aumento médio dos dias em que subiu: 153.624
